In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import simple_pe
from simple_pe.localization import event
from pesummary.utils.samples_dict import SamplesDict
from astropy.cosmology import Planck18 as cosmo
import numpy as np
from simple_pe.detectors import network
from simple_pe import io
import pandas as pd
import h5py

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


## S250818t

In [129]:
params_vals = {
        'ra': 0.32,
        'dec': -1.3,
        'iota': 1.04,
        'psi': 2.7,
        'phase': 0.7,
        'geocent_time': 1439518156.375,
        'chirp_mass_source': 11.16,
        'mass_ratio': 0.49,
        'comoving_distance': 993.61,
        'redshift': 0.24
    }

params = {key: np.array([val]) for key, val in params_vals.items()}

samples = SamplesDict(params)
samples.generate_all_posterior_samples()
samples.generate_all_posterior_samples()

2025-08-25  20:14:13 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2025-08-25  20:14:13 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.00390625Hz as default
2025-08-25  20:14:13 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  20:14:13 PESummary WARNING : Could not find f_low in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  20:14:13 PESummary WARNING : Could not find reference_frequency in input file. Using 20Hz as default
2025-08-25  20:14:13 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2025-08-25  20:14:13 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.0039062

In [130]:
ncs = io.generate_noise_curves_from_names(['AdvVirgo', 'aLIGOZeroDetHighPower'], [20]*2)
for nc in ncs.values():
    nc.generate_reach_bandwidth_vs_mchirp(grid_points=10, 
                                          mc_min = 0.95 * samples['chirp_mass'].min(),
                                          mc_max = 1.05 * samples['chirp_mass'].max())

2025-08-25  20:14:14 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default            | 0/10 [00:00<?, ?it/s]
2025-08-25  20:14:14 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  20:14:15 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default    | 1/10 [00:00<00:06,  1.30it/s]
2025-08-25  20:14:15 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  20:14:16 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default    | 2/10 [00:01<00:06,  1.33it/s]
2025-08-25  20:14:16 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2

In [131]:
ncs.update(io.generate_noise_curves_from_files([], 
                                               ['noise_curves/Asharp-asd.txt'], [10]))

In [132]:
ncs

{('AdvVirgo',
  20): <simple_pe.detectors.noise_curves.NoiseCurve at 0x76e6438cfeb0>,
 ('aLIGOZeroDetHighPower',
  20): <simple_pe.detectors.noise_curves.NoiseCurve at 0x76e6434e8640>,
 ('noise_curves/Asharp-asd.txt',
  10): <simple_pe.detectors.noise_curves.NoiseCurve at 0x76e6433c0100>}

In [133]:
netw1 = network.Network()
for ifo, psd in zip(['L1', 'H1', 'V1'], ['aLIGOZeroDetHighPower']*2 + ['AdvVirgo']):
    netw1.add_ifo_from_noise_curve(ifo, ncs[(psd,20)])
netw2 = network.Network()
for ifo, psd in zip(['L1', 'H1', 'I1'], ['aLIGOZeroDetHighPower']*3):
    netw2.add_ifo_from_noise_curve(ifo, ncs[(psd,20)])
netw3 = network.Network()
for ifo, psd in zip(['L1', 'H1', 'I1', 'V1'], ['aLIGOZeroDetHighPower']*3 + ['AdvVirgo']):
    netw3.add_ifo_from_noise_curve(ifo, ncs[(psd,20)])

In [134]:
for event_num in range(len(samples['chirp_mass'])):
    my_event1 = event.Event.from_samples_dict(samples[event_num:event_num+1])
    my_event1.add_network(netw1)

In [135]:
for event_num in range(len(samples['chirp_mass'])):
    my_event2 = event.Event.from_samples_dict(samples[event_num:event_num+1])
    my_event2.add_network(netw2)

In [136]:
for event_num in range(len(samples['chirp_mass'])):
    my_event3 = event.Event.from_samples_dict(samples[event_num:event_num+1])
    my_event3.add_network(netw3)

In [137]:
my_event1.localize_all()
my_event2.localize_all()
my_event3.localize_all()

/home/divyajyoti/miniconda3/envs/simple-pe-ngloc/lib/python3.10/site-packages/simple_pe/localization/loc.py:21: RuntimeWarning: invalid value encountered in sqrt
  sigma = 1 / np.sqrt(ev + epsilon)
/home/divyajyoti/miniconda3/envs/simple-pe-ngloc/lib/python3.10/site-packages/scipy/optimize/_zeros_py.py:806: ComplexWarning: Casting complex values to real discards the imaginary part
  r = _zeros._brentq(f, a, b, xtol, rtol, maxiter, args, full_output, disp)


In [138]:
my_event1.snr

{'L1': array([-4.17038171+6.12470861j]),
 'H1': array([3.87374607-8.95967728j]),
 'V1': array([-0.7788528+4.91430936j])}

In [139]:
my_event2.snr

{'L1': array([-4.17038171+6.12470861j]),
 'H1': array([3.87374607-8.95967728j]),
 'I1': array([1.64208179-1.33092246j])}

In [140]:
my_event3.snr

{'L1': array([-4.17038171+6.12470861j]),
 'H1': array([3.87374607-8.95967728j]),
 'I1': array([1.64208179-1.33092246j]),
 'V1': array([-0.7788528+4.91430936j])}

In [141]:
my_event1.area

{'time': 40.886623467081606,
 'coh': 29.13005477312673,
 'left': 9.776814224125408,
 'right': 11.737216709157087,
 'marg': 9.837122828749667}

In [142]:
my_event2.area

{'time': 2253.9910235387183,
 'coh': 2253.991023538718,
 'left': (1605.4450271448547+0j),
 'right': 1658.9027828786457,
 'marg': 1615.8994406272213}

In [143]:
my_event3.area

{'time': 40.886623467081606,
 'coh': 29.13005477312673,
 'left': 9.776814224125408,
 'right': 11.737216709157087,
 'marg': 9.837122828749667}